# 환경 준비

In [ ]:
# konlpy 설치하기
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 11.1MB/s 
     |████████████████████████████████| 460kB 45.9MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
# konlpy를 실행하기
import konlpy

In [ ]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [ ]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
## 문서 ID로 실제 파일 불러오기
# 28일후 2184건.csv
## https://drive.google.com/file/d/15RIakdy9V6txHyIURw87ZcczgTBe2nwL/view?usp=sharing


rawdata_downloaded = drive.CreateFile({'id': '15RIakdy9V6txHyIURw87ZcczgTBe2nwL'})
rawdata_downloaded.GetContentFile('28일후 2184건.csv')

In [ ]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('28일후 2184건.csv', sep=",")

In [ ]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,최삼식(dubi****),단순히 좀비영화로만 보면 안될 것 같습니다. 사람은 잔인하고 사람보다 무서운 동물...,10,416,44,2013-07-16 01:02:00,0
1,1,검은양말(dray****),내 생애 최고의 좀비영화. 영화음악은 정말 최고임.,10,292,36,2013-06-26 16:52:00,0
2,2,superb(lsy7****),이 영화의 가장 마음에 드는건 영화 자체에 깔려있는 어두운 느낌. 이 영화에서 좀비...,9,250,17,2012-12-30 17:51:00,0
3,3,사과(rla9****),이영화가 7점대라니 말도안된다. 적어도 8.5는 넘어야될 수작임. 안보신분들은 보셈...,10,222,28,2013-11-13 23:39:00,0
4,4,atoz****,이때 까지 나왔던 좀비영화중 최고의 걸작,10,201,26,2013-07-06 22:25:00,0
...,...,...,...,...,...,...,...,...
2175,2175,컵라면맛라면(tjdj****),개 노잼 쓰레기 영화. 주인공이 너무 발암임. 저예산으로 너무 많은걸 보여주려함ㅡㅡ,1,2,9,2017-08-01 05:59:00,0
2176,2176,HJ(jiho****),개.노.잼 알바들 믿지마세요,1,2,10,2018-08-18 05:36:00,0
2177,2177,번뜩임(elha****),"후반부 주인공이 뜬금없이 슈퍼맨 되버리고, 지 하나 살자고 좀비 풀어주는거 어이가...",1,1,10,2019-06-04 00:23:00,0
2178,2178,몰랑이말랑이(cher****),예상보다 진짜 별로네.. 좀비영화 안본게없지만.. 엄청 기대를했는데 실망이네.. 왜...,1,3,12,2019-06-30 17:20:00,0


In [ ]:
원본데이터NAN삭제 = 원본데이터.dropna(axis=0)
## 바로: NAN값(빈값)을 삭제하기 위한 프로세스

In [ ]:
원본데이터NAN삭제

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,최삼식(dubi****),단순히 좀비영화로만 보면 안될 것 같습니다. 사람은 잔인하고 사람보다 무서운 동물...,10,416,44,2013-07-16 01:02:00,0
1,1,검은양말(dray****),내 생애 최고의 좀비영화. 영화음악은 정말 최고임.,10,292,36,2013-06-26 16:52:00,0
2,2,superb(lsy7****),이 영화의 가장 마음에 드는건 영화 자체에 깔려있는 어두운 느낌. 이 영화에서 좀비...,9,250,17,2012-12-30 17:51:00,0
3,3,사과(rla9****),이영화가 7점대라니 말도안된다. 적어도 8.5는 넘어야될 수작임. 안보신분들은 보셈...,10,222,28,2013-11-13 23:39:00,0
4,4,atoz****,이때 까지 나왔던 좀비영화중 최고의 걸작,10,201,26,2013-07-06 22:25:00,0
...,...,...,...,...,...,...,...,...
2175,2175,컵라면맛라면(tjdj****),개 노잼 쓰레기 영화. 주인공이 너무 발암임. 저예산으로 너무 많은걸 보여주려함ㅡㅡ,1,2,9,2017-08-01 05:59:00,0
2176,2176,HJ(jiho****),개.노.잼 알바들 믿지마세요,1,2,10,2018-08-18 05:36:00,0
2177,2177,번뜩임(elha****),"후반부 주인공이 뜬금없이 슈퍼맨 되버리고, 지 하나 살자고 좀비 풀어주는거 어이가...",1,1,10,2019-06-04 00:23:00,0
2178,2178,몰랑이말랑이(cher****),예상보다 진짜 별로네.. 좀비영화 안본게없지만.. 엄청 기대를했는데 실망이네.. 왜...,1,3,12,2019-06-30 17:20:00,0


# 형태소 분석

In [ ]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [ ]:
# "원본데이터NAN삭제"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터NAN삭제.iterrows(): # 원본데이터NAN삭제의 컬럼(행)을 불러와서 반복할 준비!
    대상텍스트 = row[2] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 대상 텍스트의 열은 2번입니다. python은 0부터 숫자를 세게 됩니다!!!
    분류 = row[3] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 평점 정보는 3번째 열에 있습니다.
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [ ]:
# "형태소" 변수의 저장 내용 확인
형태소

[('딱히', 'Adjective'),
 ('좀비', 'Noun'),
 ('영화', 'Noun'),
 ('인지', 'Josa'),
 ('모르겟는건', 'Verb'),
 ('나', 'Noun'),
 ('만', 'Josa'),
 ('그', 'Determiner'),
 ('런가', 'Noun'),
 ('..', 'Punctuation')]

In [ ]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,딱히,Adjective,1,1
1,좀비,Noun,1,1
2,영화,Noun,1,1
3,인지,Josa,1,1
4,모르겟는건,Verb,1,1
5,나,Noun,1,1
6,만,Josa,1,1
7,그,Determiner,1,1
8,런가,Noun,1,1
9,..,Punctuation,1,1


In [ ]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[       0            1  type  count
 0    단순히    Adjective    10      1
 1     좀비         Noun    10      1
 2    영화로         Noun    10      1
 3      만         Josa    10      1
 4     보면         Verb    10      1
 5     안될    Adjective    10      1
 6      것         Noun    10      1
 7   같습니다    Adjective    10      1
 8      .  Punctuation    10      1
 9     사람         Noun    10      1
 10     은         Josa    10      1
 11  잔인하고    Adjective    10      1
 12    사람         Noun    10      1
 13    보다         Josa    10      1
 14   무서운    Adjective    10      1
 15    동물         Noun    10      1
 16     은         Josa    10      1
 17    없다    Adjective    10      1
 18     .  Punctuation    10      1
 19     좀         Noun    10      1
 20   심오한    Adjective    10      1
 21    의미         Noun    10      1
 22     가         Josa    10      1
 23  담겨있는         Verb    10      1
 24    영화         Noun    10      1
 25   입니다    Adjective    10      1
 26     .  Punctuation    10

In [ ]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,단순히,Adjective,10,1
1,좀비,Noun,10,1
2,영화로,Noun,10,1
3,만,Josa,10,1
4,보면,Verb,10,1
...,...,...,...,...
5,나,Noun,1,1
6,만,Josa,1,1
7,그,Determiner,1,1
8,런가,Noun,1,1


In [ ]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,단순히,Adjective,10,1
1,좀비,Noun,10,1
2,영화로,Noun,10,1
3,만,Josa,10,1
4,보면,Verb,10,1
...,...,...,...,...
5,나,Noun,1,1
6,만,Josa,1,1
7,그,Determiner,1,1
8,런가,Noun,1,1


In [ ]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소    품사           분류
!      Punctuation  1     7
                    2     1
                    3     1
                    6     1
                    7     8
                         ..
힘들다    Adjective    8     1
힘들었던   Adjective    10    1
힘들었어요  Adjective    3     1
힘듬     Adjective    10    1
힘없는    Adjective    8     1
Name: 카운트, Length: 9808, dtype: int64

In [ ]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소   품사          분류     
!     Punctuation 1     7
                  2     1
                  3     1
                  6     1
                  7     8
...                   ...
힘들다   Adjective   8     1
힘들었던  Adjective   10    1
힘들었어요 Adjective   3     1
힘듬    Adjective   10    1
힘없는   Adjective   8     1

[9808 rows x 1 columns]

In [ ]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')